In [ ]:
# Create a new table for GPT generated descriptions (!ONLY NECESSARY FOR HAVEN STAGING DB!)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE gpt_description (
  DescriptionID INT AUTO_INCREMENT PRIMARY KEY,
  Description VARCHAR(255),
  TokensUsed INT,
  id INT,
  CreatedOn DATETIME,
  UpdatedOn DATETIME,
  FOREIGN KEY (id) REFERENCES opportunity(id)
);
""")

In [ ]:
# Create a new table for GPT generated keywords (!ONLY NECESSARY FOR HAVEN STAGING DB!)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE gpt_keyword (
  KeywordID INT AUTO_INCREMENT PRIMARY KEY,
  Keyword VARCHAR(255) NOT NULL,
  TokensUsed INT,
  id INT,
  CreatedOn DATETIME,
  UpdatedOn DATETIME,
  FOREIGN KEY (id) REFERENCES opportunity(id)
);
""")

In [5]:
import mysql.connector
import openai
from bs4 import BeautifulSoup
from getpass import getpass
from datetime import datetime

# Connect to the MySQL database
conn = mysql.connector.connect(host='tylerathomas0.lmu.build', database='tylerath_opportunities', user='tylerath_opps', password='NvX5EAfvtr4KRBx')
cursor = conn.cursor()

# Set up OpenAI API credentials
openai.api_key = getpass()

# Retrieve the description column data
cursor.execute("SELECT id, description FROM opportunity LIMIT 10")
descriptions = cursor.fetchall()

# Loop through each description, extract the text using BeautifulSoup, and generate a summary using OpenAI API
for opportunity_id, description in descriptions:
    html = description
    # BS description here
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text()
    truncated_text = text[:15000]
    # Implementing chatgpt api
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "You summarize opportunities from various government industries."},
            {"role": "user", "content": f'Summarize the contract work detail in the following text.: "{truncated_text}"'}
        ],
        temperature=0,
    )

    summary = response['choices'][0]['message']['content']
    tokens = response['usage']['total_tokens']
    print(summary)
    print(tokens)

    # Insert cleaned text and summary into 'bsdesc' and 'chatgptdesc' columns, respectively
    cursor.execute("""
        INSERT INTO gpt_description (Description, TokensUsed, id, CreatedOn, UpdatedOn)
        VALUES (%s, %s, %s, %s, %s)
    """, (summary, tokens, opportunity_id, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

    # Commit changes to the database
    conn.commit()

# Close database connection
conn.close()

········
The government is seeking bids for a contract to supply barricades for a special event. The materials required include standard 1" thick x 8" wide x 16' long barricade boards with reflective white and orange striping on both sides of the board, as well as barricade legs, plastic "A" frame type. The crossbar should be wooden, not plastic molded, and engineering grade for reflective striping. The delivery deadline is December 1, 2022, to the City of Pasadena, Public Works Yards. The estimated bid value is not provided, and the actual purchases will be made at the department's sole discretion. All questions should be submitted online.
652
The government is looking to procure 12 units of NSN 4710016990074 TUBE ASSEMBLY, METAL. The delivery location is W1A8 DLA DISTRIBUTION, and the delivery is expected within 0175 days. The approved source for this procurement is 1KZK6 UPM070720. The solicitation is an RFQ, and interested parties can submit their quotes electronically. There are n

KeyboardInterrupt: 

In [4]:
import mysql.connector
import os
import openai
from getpass import getpass
from datetime import datetime

# Connect to the MySQL database
conn = mysql.connector.connect(host='tylerathomas0.lmu.build', database='tylerath_opportunities', user='tylerath_opps', password='NvX5EAfvtr4KRBx')
cursor = conn.cursor()

openai.api_key = getpass()

# Retrieve the data from the 'gpt_description' table
cursor.execute("SELECT DescriptionID, Description, TokensUsed, id FROM gpt_description")
descriptions = cursor.fetchall()

# Loop through each description and generate keywords using OpenAI gpt-3.5-turbo model
for desc in descriptions:
    cleanDesc = desc[1]
    response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        #system predefined message. can be used to define number of characters and whatnot. Telling the model what it is.
        {"role": "system", "content": "You are an assistant generating keywords for government contracting descriptions."},
        #what a user would type in, using it as our instruction
        {"role": "user", "content": f'Generate keywords for the following prompt. Keywords should relate ONLY to the work detail of the project involved. Ignore any names, dates, and emails. Separate keywords by commas: "{cleanDesc}"'}
    ],
    temperature=0,
    )
    keywords = response['choices'][0]['message']['content']
    tokens = response['usage']['total_tokens']
    keywords_list = [keyword.strip() for keyword in keywords.split(',')]  # Delimit and split the keywords by commas
    print(keywords_list)
    print(tokens)
    
    # Insert the generated keyword data into the 'gpt_keyword' table
    for keyword in keywords_list:
        cursor.execute(
            "INSERT INTO gpt_keyword (Keyword, TokensUsed, id, CreatedOn, UpdatedOn) "
            "VALUES (%s, %s, %s, %s, %s)",
            (keyword, tokens, desc[3], datetime.now().strftime('%Y-%m-%d %H:%M:%S'), datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        )
    
    # commit changes to the database
    conn.commit()

conn.close()

········
['government contracting', 'bids', 'contract', 'supply', 'barricades', 'special event', 'materials', 'standard', '1" thick', '8" wide', "16' long", 'barricade boards', 'reflective', 'white', 'orange', 'striping', 'both sides.']
166
['government contracting', 'procurement', 'NSN 4710016990074', 'tube assembly', 'metal', 'delivery', 'DLA Distribution', 'approved source', '1KZK6 UPM070720', 'solicitation.']
174
['government contracting', 'procurement', 'observation window', 'NSN', 'delivery location', 'delivery deadline', 'approved sources.']
145
['Cold Spring Construction Co.', 'bid', 'contract', 'Allegany County Department of Public Works', 'Pine Street Bridge', 'Canacadea Creek', 'Town of Alfred', 'Village of Alfred', 'Allegany County', 'replacing', 'construction', 'infrastructure', 'bridge replacement', 'engineering', 'transportation', 'civil engineering', 'construction project.']
170
['contract work', 'repairing', 'replacing', 'backflow systems', 'National Mall', 'Memorial P